In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
basepath = Path('.')
rawpath = basepath/'raw'
datapath = basepath/'data'
datapath.mkdir(exist_ok=True)

# Merge Raw

In [4]:
csvfiles = rawpath.glob('*.csv')
df_list = []

In [5]:
for csvfile in csvfiles:
    tmp = pd.read_csv(csvfile, skiprows=2, na_values='-')
    tmp.dropna(thresh=8, inplace=True)
    df_list.append(tmp)

In [6]:
df = pd.concat(df_list)

In [7]:
df.drop('S/N',axis=1, inplace=True)

In [8]:
df.head()

,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale
0,1 KING ALBERT PARK,KING ALBERT PARK,Condominium,21,RCR,Freehold,Resale,1.0,1668888.0,NaN,1173.0,Strata,01 to 05,1422.0,Sep-2019
1,# 1 LOFT,LORONG 24 GEYLANG,Apartment,14,RCR,Freehold,Resale,1.0,675000.0,NaN,570.0,Strata,01 to 05,1183.0,Sep-2019
2,1 KING ALBERT PARK,KING ALBERT PARK,Condominium,21,RCR,Freehold,Resale,1.0,1900000.0,NaN,1281.0,Strata,01 to 05,1483.0,Aug-2019
3,1 KING ALBERT PARK,KING ALBERT PARK,Condominium,21,RCR,Freehold,Resale,1.0,1880000.0,NaN,1281.0,Strata,01 to 05,1468.0,Jul-2019
4,1 CANBERRA,CANBERRA DRIVE,Executive Condominium,27,OCR,99 yrs lease commencing from 2012,Resale,1.0,845000.0,NaN,958.0,Strata,01 to 05,882.0,Jul-2019


# Calculate Present Value

In [9]:
indices_df = pd.read_csv(datapath/'PropertyPriceIndex.csv')

In [10]:
price_index = indices_df['Index Value']
price_index.index = pd.MultiIndex.from_arrays(indices_df[['Year', 'Quarter']].values.T, names=['Year', 'Quarter'])

In [11]:
def calc_present_value(datestring, price):
    tmp = datestring.split('-')
    month, year = tmp[0], int(tmp[1])
    
    if month in ('Jan','Feb','Mar'):
        quarter = 1
    elif month in ('Apr','May','Jun'):
        quarter = 2
    elif month in ('Jul','Aug','Sep'):
        quarter = 3
    else:
        quarter = 4
    
    latest = price_index.index[-1]
    
    if not (year == latest[0] and quarter == latest[1]):
        pv = price/price_index[year, quarter] * price_index[latest[0],latest[1]]
        return pv.round(2)
    else:
        return price

In [12]:
df['Present Value'] = df.apply(lambda x: calc_present_value(x['Date of Sale'], x['Price ($)']), axis=1)

In [13]:
df[['Date of Sale','Price ($)', 'Present Value']].head(10)

,Date of Sale,Price ($),Present Value
0,Sep-2019,1668888.0,1668888.00
1,Sep-2019,675000.0,675000.00
2,Aug-2019,1900000.0,1900000.00
3,Jul-2019,1880000.0,1880000.00
4,Jul-2019,845000.0,845000.00
5,Jul-2019,2180000.0,2180000.00
6,Jul-2019,2350000.0,2350000.00
7,Jun-2019,668000.0,674201.59
8,May-2019,980000.0,989098.14
9,Apr-2019,960000.0,968912.47


# Output Data

In [14]:
df.to_csv(datapath/'merged_data.csv', index=False)